Extracting Data From Yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import datetime
from datetime import date, timedelta

In [ ]:
# Select Stocks
print("Select the stocks by entering SNo:\n1: TCS.NS  2: RELIANCE.NS  3: INFY.NS")
a = int(input())
if a == 1:
    stock = "TCS.NS"
elif a == 2:
    stock = "RELIANCE.NS"
elif a == 3:
    stock = "INFY.NS"
else:
  print("Enter a Valid SNo")
today = date.today()
d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=300)
d2.strftime("%Y-%m-%d")
start_date = d2
data = yf.download(stock, start=start_date, end=end_date, progress=False, auto_adjust=True, group_by='column')
data.columns = data.columns.droplevel(1)


Select the stocks by entering SNo:
1: TCS.NS  2: RELIANCE.NS  3: INFY.NS
1


In [ ]:
print(data)

Price             Close         High          Low         Open   Volume
Date                                                                   
2024-09-02  4400.170898  4465.331052  4391.363058  4453.651895  1216500
2024-09-03  4391.703613  4434.089029  4375.693246  4400.170717  1717263
2024-09-04  4359.488770  4367.031559  4318.076614  4357.250454  1265505
2024-09-05  4356.277344  4378.710869  4335.059922  4361.192133  1688793
2024-09-06  4337.590332  4398.127487  4321.288175  4354.330647  1649179
...                 ...          ...          ...          ...      ...
2025-06-20  3435.699951  3446.300049  3410.899902  3425.000000  2351248
2025-06-23  3393.600098  3425.000000  3370.000000  3415.000000  3123421
2025-06-24  3390.399902  3442.100098  3382.000000  3434.600098  3403048
2025-06-25  3444.800049  3450.000000  3400.000000  3407.300049  1757699
2025-06-26  3441.800049  3451.399902  3415.300049  3444.800049  2879106

[204 rows x 5 columns]


In [ ]:
data["Date"] = data.index
data = data[["Date","Open","Close","High","Low","Volume"]]
data.reset_index(drop=True,inplace = True)

data.index.name = "SNo"

In [ ]:
print(data)

Price       Date         Open        Close         High          Low   Volume
SNo                                                                          
0     2024-09-02  4453.651895  4400.170898  4465.331052  4391.363058  1216500
1     2024-09-03  4400.170717  4391.703613  4434.089029  4375.693246  1717263
2     2024-09-04  4357.250454  4359.488770  4367.031559  4318.076614  1265505
3     2024-09-05  4361.192133  4356.277344  4378.710869  4335.059922  1688793
4     2024-09-06  4354.330647  4337.590332  4398.127487  4321.288175  1649179
..           ...          ...          ...          ...          ...      ...
199   2025-06-20  3425.000000  3435.699951  3446.300049  3410.899902  2351248
200   2025-06-23  3415.000000  3393.600098  3425.000000  3370.000000  3123421
201   2025-06-24  3434.600098  3390.399902  3442.100098  3382.000000  3403048
202   2025-06-25  3407.300049  3444.800049  3450.000000  3400.000000  1757699
203   2025-06-26  3444.800049  3441.800049  3451.399902  3415.30

Trading Strategy

In [ ]:
df = data

# Dma for 20 days
df['20dma'] = df['Close'].rolling(20).mean()
# Dma for 50 days
df['50dma'] = df['Close'].rolling(50).mean()

# RSI
delta = df['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
rs = gain / loss
df['rsi'] = 100 - (100 / (1 + rs))

# MACD
ema_short = df['Close'].ewm(span=12, adjust=False).mean()
ema_long = df['Close'].ewm(span=26, adjust=False).mean()


df['macd'] = ema_short - ema_long

df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()

df['macd_hist'] = df['macd'] - df['macd_signal']

dd = df[['Close', '20dma', '50dma', 'rsi']]
print(df)

Price       Date         Open        Close         High          Low   Volume  \
SNo                                                                             
0     2024-09-02  4453.651895  4400.170898  4465.331052  4391.363058  1216500   
1     2024-09-03  4400.170717  4391.703613  4434.089029  4375.693246  1717263   
2     2024-09-04  4357.250454  4359.488770  4367.031559  4318.076614  1265505   
3     2024-09-05  4361.192133  4356.277344  4378.710869  4335.059922  1688793   
4     2024-09-06  4354.330647  4337.590332  4398.127487  4321.288175  1649179   
..           ...          ...          ...          ...          ...      ...   
199   2025-06-20  3425.000000  3435.699951  3446.300049  3410.899902  2351248   
200   2025-06-23  3415.000000  3393.600098  3425.000000  3370.000000  3123421   
201   2025-06-24  3434.600098  3390.399902  3442.100098  3382.000000  3403048   
202   2025-06-25  3407.300049  3444.800049  3450.000000  3400.000000  1757699   
203   2025-06-26  3444.80004

Genrating Signals

In [ ]:
#writing Signals
df['signal'] = 'HOLD'

# Buy signal
buy = (df['rsi'] < 30) & (df['20dma'] > df['50dma'])
df.loc[buy, 'signal'] = 'BUY'

#sell Signal

sell = (df['rsi'] > 70) & (df['20dma'] < df['50dma'])
df.loc[sell, 'signal'] = 'SELL'


trade_df = df[df['signal'] != 'HOLD']
trades = []
for index, row in trade_df.iterrows():
    trades.append({
        'Date': row['Date'].strftime('%Y-%m-%d'),
        'Company': stock,
        'Signal': row['signal'],
        'Price': row['Close']
    })



Creating Google Sheet

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving my-project-trading-464114-d247896726b7.json to my-project-trading-464114-d247896726b7.json


In [ ]:
import gspread
from google.auth.transport.requests import Request
from google.colab import auth
from google.oauth2.service_account import Credentials

creds_json_path = 'my-project-trading-464114-d247896726b7.json'
sheet_name = 'Algo_Trading'

auth.authenticate_user()

scopes = [
      'https://www.googleapis.com/auth/spreadsheets',
      'https://www.googleapis.com/auth/drive'
  ]

creds = Credentials.from_service_account_file(creds_json_path, scopes=scopes)
client = gspread.authorize(creds)
sheet = client.open(sheet_name)


#checking is worksheet is their

try:
    worksheet = sheet.worksheet('Sheet1')
    sheet.del_worksheet(worksheet)
except gspread.exceptions.WorksheetNotFound:
    pass
# adding Data to the worksheet
worksheet = sheet.add_worksheet(title='Sheet1', rows=1000, cols=10)
worksheet.append_row(['Date', 'Company', 'Signal', 'Price'])
rows = [[t['Date'], t['Company'], t['Signal'], t['Price']] for t in trades]
rows = []
for t in trades:
    date_str = str(t['Date'])
    rows.append([date_str, t['Company'], t['Signal'], t['Price']])

worksheet.update(values=rows, range_name=f'A2:D{len(rows)+1}')



print("Trades Saved")
print("Sheet URL:", f"https://docs.google.com/spreadsheets/d/{sheet.id}")

Trades Saved
Sheet URL: https://docs.google.com/spreadsheets/d/1FM3dqCgNmDfvoho3MmySX7O7cgU4XqJ1N7VAlQwagMs


In [ ]:
df['target'] = (df['Close'].shift(-1) > df['Close']).astype(int)

Machine Learing Model and Prediction

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

features = ['rsi', '20dma', '50dma', 'Volume', 'macd', 'macd_signal', 'macd_hist']

# Drop rows with missing features or target
df = df.dropna(subset=features + ['target'])

X = df[features]
y = df['target']

# Impute missing values
imputer = SimpleImputer(strategy='mean')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=features)


X_train, X_test, y_train, y_test = train_test_split(
    X_imputed, y, test_size=0.2, shuffle=True, random_state=42
)


scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=features)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=features)

# Logistic Regression with higher max_iter
model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
model.fit(X_train_scaled, y_train)

# Predict and evaluate accuracy
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)

latest_features = df[features].iloc[-1:].copy()


latest_imputed = pd.DataFrame(imputer.transform(latest_features), columns=features)
latest_scaled = pd.DataFrame(scaler.transform(latest_imputed), columns=features)

# Predict next day movement and probability
future_pred = model.predict(latest_scaled)[0]
future_prob = model.predict_proba(latest_scaled)[0, 1]

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on test set: {accuracy*100:.4f}")


print(f"\nFuture prediction (next day): {'Up' if future_pred == 1 else 'Down'}")

Accuracy on test set: 58.0645

Future prediction (next day): Up
